https://www.youtube.com/watch?v=Rp5LWT4or-o&list=PLxy0DxWEupiODTF_xM5Lw1ghc0XtLCUhC&index=1

In [5]:
import findspark
findspark.init()

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import DataFrame
from pyspark.sql.window import Window

In [7]:
spark = SparkSession.builder.appName('GC').getOrCreate()
spark

### 13 Write a Pyspark query to report the movies with an odd-numbered ID and a description that is not 'boring'. Return the result table in descending order by rating.

In [5]:
data=[(1, 'War','great 3D',8.9) 
,(2, 'Science','fiction',8.5) 
,(3, 'irish','boring',6.2)
,(4, 'Ice song','Fantacy',8.6)  
,(5, 'House card','Interesting',9.1)]   
schema="ID int,movie string,description string,rating double"
df=spark.createDataFrame(data,schema) 
df.show() 

+---+----------+-----------+------+
| ID|     movie|description|rating|
+---+----------+-----------+------+
|  1|       War|   great 3D|   8.9|
|  2|   Science|    fiction|   8.5|
|  3|     irish|     boring|   6.2|
|  4|  Ice song|    Fantacy|   8.6|
|  5|House card|Interesting|   9.1|
+---+----------+-----------+------+



In [6]:
df.where((col('ID') % 2 != 0 ) & (trim(lower(col('description'))) != 'boring')).orderBy(col('rating').desc()).show()

+---+----------+-----------+------+
| ID|     movie|description|rating|
+---+----------+-----------+------+
|  5|House card|Interesting|   9.1|
|  1|       War|   great 3D|   8.9|
+---+----------+-----------+------+



In [7]:
# OR 
df.select('*').filter((col('ID')%2!=0) & (trim(lower(col('description'))) != 'boring')).orderBy(col('rating').desc()).show()

+---+----------+-----------+------+
| ID|     movie|description|rating|
+---+----------+-----------+------+
|  5|House card|Interesting|   9.1|
|  1|       War|   great 3D|   8.9|
+---+----------+-----------+------+



### 14 Collect_list and Aggregation

In [13]:
data = [
    ("john", "tomato", 2),
    ("bill", "apple", 2),
    ("john", "banana", 2),
    ("john", "tomato", 3),
    ("bill", "taco", 2),
    ("bill", "apple", 2),
]
schema = "name string,item string,weight int"
df = spark.createDataFrame(data, schema)
df.show()

+----+------+------+
|name|  item|weight|
+----+------+------+
|john|tomato|     2|
|bill| apple|     2|
|john|banana|     2|
|john|tomato|     3|
|bill|  taco|     2|
|bill| apple|     2|
+----+------+------+



In [14]:
df_final =df.groupBy('name', 'item').agg(sum(col('weight')).alias('sum_weight'))
df_final.show()

+----+------+----------+
|name|  item|sum_weight|
+----+------+----------+
|john|tomato|         5|
|bill| apple|         4|
|john|banana|         2|
|bill|  taco|         2|
+----+------+----------+



In [15]:
df_fi = df_final.groupBy('name').agg(collect_list(struct('item', 'sum_weight')).alias('items')).orderBy('name')
df_fi.show(truncate= False)

+----+--------------------------+
|name|items                     |
+----+--------------------------+
|bill|[{apple, 4}, {taco, 2}]   |
|john|[{tomato, 5}, {banana, 2}]|
+----+--------------------------+



### 15 find all duplicate emails
Write a pyspark dataframe query to find all duplicate emails in a table named Person.

In [16]:
data = [(1, "abc@gmail.com"), (2, "bcd@gmail.com"), (3, "abc@gmail.com")]
schema = "ID int,email string"
df = spark.createDataFrame(data, schema)
df.show()

+---+-------------+
| ID|        email|
+---+-------------+
|  1|abc@gmail.com|
|  2|bcd@gmail.com|
|  3|abc@gmail.com|
+---+-------------+



In [17]:
df.groupBy('email').count().filter(col('count')>1).select('email').show()

+-------------+
|        email|
+-------------+
|abc@gmail.com|
+-------------+



In [18]:
df.createOrReplaceTempView('test')
spark.sql("""select email from test group by email having count(email)>1""").show()

+-------------+
|        email|
+-------------+
|abc@gmail.com|
+-------------+



### 18 Write a pyspark query for a report that provides the customer ids from the customer table that bought all the products in the product table.

In [8]:
data=[(1,5),(2,6),(3,5),(3,6),(1,6)]
schema="customer_id int,product_key int"
customer_df=spark.createDataFrame(data,schema)

data=[(5,),(6,)]
schema="product_key int"
product_df=spark.createDataFrame(data,schema)

customer_df.show()
product_df.show()

+-----------+-----------+
|customer_id|product_key|
+-----------+-----------+
|          1|          5|
|          2|          6|
|          3|          5|
|          3|          6|
|          1|          6|
+-----------+-----------+

+-----------+
|product_key|
+-----------+
|          5|
|          6|
+-----------+



In [9]:
# take unique key from product table 
df_product = product_df.agg(countDistinct(col('product_key')).alias('cnt_products'))
df_product.show()

+------------+
|cnt_products|
+------------+
|           2|
+------------+



In [10]:
df_customer = customer_df.groupBy(col('customer_id')).agg(countDistinct(col('product_key')).alias('cnt_products'))
df_customer.show()

+-----------+------------+
|customer_id|cnt_products|
+-----------+------------+
|          1|           2|
|          3|           2|
|          2|           1|
+-----------+------------+



In [11]:
df = df_product.join(df_customer, df_product.cnt_products == df_customer.cnt_products, 'inner').select('customer_id')
df.show()

+-----------+
|customer_id|
+-----------+
|          1|
|          3|
+-----------+



### 19 . Get the employees dept id with maximun and minimum salary in each department

In [42]:
data=[('Genece' , 2 , 75000),
('Jaimin' , 2 , 80000 ),
('Pankaj' , 2 , 80000 ),
('Tarvares' , 2 , 70000),
('Marlania' , 4 , 70000),
('Briana' , 4 , 85000),
('Kimberi' , 4 , 55000),
('Gabriella' , 4 , 55000),  
('Lakken', 5, 60000),
('Latoynia' , 5 , 65000) ]
schema="emp_name string,dept_id int,salary int"
df=spark.createDataFrame(data,schema)
df.show()

+---------+-------+------+
| emp_name|dept_id|salary|
+---------+-------+------+
|   Genece|      2| 75000|
|   Jaimin|      2| 80000|
|   Pankaj|      2| 80000|
| Tarvares|      2| 70000|
| Marlania|      4| 70000|
|   Briana|      4| 85000|
|  Kimberi|      4| 55000|
|Gabriella|      4| 55000|
|   Lakken|      5| 60000|
| Latoynia|      5| 65000|
+---------+-------+------+



In [49]:
df_1 = df.groupBy('dept_id').agg(max('salary').alias('max_salary'), min('salary').alias('min_salary')).withColumnRenamed('dept_id', 'dept_id_1')
df_1.show()

+---------+----------+----------+
|dept_id_1|max_salary|min_salary|
+---------+----------+----------+
|        2|     80000|     70000|
|        4|     85000|     55000|
|        5|     65000|     60000|
+---------+----------+----------+



In [51]:
df_2 = df.join(df_1, df.dept_id == df_1.dept_id_1, 'inner')
df_2.show()

+---------+-------+------+---------+----------+----------+
| emp_name|dept_id|salary|dept_id_1|max_salary|min_salary|
+---------+-------+------+---------+----------+----------+
|   Genece|      2| 75000|        2|     80000|     70000|
|   Jaimin|      2| 80000|        2|     80000|     70000|
|   Pankaj|      2| 80000|        2|     80000|     70000|
| Marlania|      4| 70000|        4|     85000|     55000|
| Tarvares|      2| 70000|        2|     80000|     70000|
|   Briana|      4| 85000|        4|     85000|     55000|
|  Kimberi|      4| 55000|        4|     85000|     55000|
|Gabriella|      4| 55000|        4|     85000|     55000|
|   Lakken|      5| 60000|        5|     65000|     60000|
| Latoynia|      5| 65000|        5|     65000|     60000|
+---------+-------+------+---------+----------+----------+



In [52]:
df_3 = df_2.filter((col('max_salary') == col('salary')) | (col('min_salary') == col('salary')))
df_3.show()

+---------+-------+------+---------+----------+----------+
| emp_name|dept_id|salary|dept_id_1|max_salary|min_salary|
+---------+-------+------+---------+----------+----------+
|   Jaimin|      2| 80000|        2|     80000|     70000|
|   Pankaj|      2| 80000|        2|     80000|     70000|
| Tarvares|      2| 70000|        2|     80000|     70000|
|   Briana|      4| 85000|        4|     85000|     55000|
|  Kimberi|      4| 55000|        4|     85000|     55000|
|Gabriella|      4| 55000|        4|     85000|     55000|
|   Lakken|      5| 60000|        5|     65000|     60000|
| Latoynia|      5| 65000|        5|     65000|     60000|
+---------+-------+------+---------+----------+----------+



In [53]:
df_f = df_3.groupBy('dept_id', 'salary').agg(collect_list('emp_name').alias('emp_names'))
df_f.show()

+-------+------+--------------------+
|dept_id|salary|           emp_names|
+-------+------+--------------------+
|      2| 80000|    [Jaimin, Pankaj]|
|      2| 70000|          [Tarvares]|
|      4| 85000|            [Briana]|
|      4| 55000|[Kimberi, Gabriella]|
|      5| 60000|            [Lakken]|
|      5| 65000|          [Latoynia]|
+-------+------+--------------------+



### 20 . Cache and Persist DataFrame

In [65]:
data = [("Raj","Doe",None),
 (None,"Samuel","VIZAG"),
 ("David","Smith", None),
 ("Samson",None, "HYD"),
 ("Immi", "Steve", "BNG"),
 (None, None, None)]

columns = ["Firstname", "Lastname", "City"]

df = spark.createDataFrame(data,columns)
df.cache()
df.show()

+---------+--------+-----+
|Firstname|Lastname| City|
+---------+--------+-----+
|      Raj|     Doe| NULL|
|     NULL|  Samuel|VIZAG|
|    David|   Smith| NULL|
|   Samson|    NULL|  HYD|
|     Immi|   Steve|  BNG|
|     NULL|    NULL| NULL|
+---------+--------+-----+



In [66]:
for i in df.columns:
    total_count = df.select(col(i)).count()
    null_values = df.filter(col(i).isNull()).count()
    percentage = (null_values/total_count)*100
    print(i, percentage)

Firstname 33.33333333333333
Lastname 33.33333333333333
City 50.0


### 25

In [16]:
data = [('Alice' ,['Badminton', 'Tennis']),
        ('Bob', ['Tennis', 'Cricket']),
        ('Julie', ['Cricket','Carroms'])]
schema = ['name', 'Hobbies']
df = spark.createDataFrame(data=data, schema= schema)
df.show()
df.printSchema()

+-----+-------------------+
| name|            Hobbies|
+-----+-------------------+
|Alice|[Badminton, Tennis]|
|  Bob|  [Tennis, Cricket]|
|Julie| [Cricket, Carroms]|
+-----+-------------------+

root
 |-- name: string (nullable = true)
 |-- Hobbies: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [17]:
df.select(col('name'), explode(col('Hobbies'))).show()

+-----+---------+
| name|      col|
+-----+---------+
|Alice|Badminton|
|Alice|   Tennis|
|  Bob|   Tennis|
|  Bob|  Cricket|
|Julie|  Cricket|
|Julie|  Carroms|
+-----+---------+



In [18]:
data = [('Goa','','AP'), ('','AP',None),(None, '','Blr')]
schema = 'City1 string, City2 string, City3 string'
df2 = spark.createDataFrame(data, schema)
df2.show()

+-----+-----+-----+
|City1|City2|City3|
+-----+-----+-----+
|  Goa|     |   AP|
|     |   AP| NULL|
| NULL|     |  Blr|
+-----+-----+-----+



In [25]:
df_2 = df2.withColumn('City1', when(col('City1')=='',None).otherwise(col('City1')))\
        .withColumn('City2', when(col('City2')=='',None).otherwise(col('City2')))\
        .withColumn('City3', when(col('City3')=='',None).otherwise(col('City3')))
df_2.show()

+-----+-----+-----+
|City1|City2|City3|
+-----+-----+-----+
|  Goa| NULL|   AP|
| NULL|   AP| NULL|
| NULL| NULL|  Blr|
+-----+-----+-----+



In [28]:
dff= df_2.withColumn('Result', coalesce(col('City1'), col('City2'), col('City3'))).select('Result')
dff.show()

+------+
|Result|
+------+
|   Goa|
|    AP|
|   Blr|
+------+



### 26

In [39]:
data=[('John Doe','{"street": "123 Main St", "city": "Anytown"}'),('Jane Smith','{"street": "456 Elm St", "city": "Othertown"}')]
df=spark.createDataFrame(data,schema="name string,address string")
df.show(truncate=False)

+----------+---------------------------------------------+
|name      |address                                      |
+----------+---------------------------------------------+
|John Doe  |{"street": "123 Main St", "city": "Anytown"} |
|Jane Smith|{"street": "456 Elm St", "city": "Othertown"}|
+----------+---------------------------------------------+



In [40]:
df1 = df.withColumn('parsed_json', from_json(col('address'),'street string, city string'))
df1.show(truncate= False)

+----------+---------------------------------------------+-----------------------+
|name      |address                                      |parsed_json            |
+----------+---------------------------------------------+-----------------------+
|John Doe  |{"street": "123 Main St", "city": "Anytown"} |{123 Main St, Anytown} |
|Jane Smith|{"street": "456 Elm St", "city": "Othertown"}|{456 Elm St, Othertown}|
+----------+---------------------------------------------+-----------------------+



In [41]:
df2 = df1.withColumn(col('name'), col('parsed_json').street.alias('street'),  col('parsed_json').city.alias('city'))
df2.show()

TypeError: DataFrame.withColumn() takes 3 positional arguments but 4 were given

### 27

In [22]:
df = spark.createDataFrame(data=[(1,'A'), (2,'B')], schema="Id int, Name string")
df.show()

+---+----+
| Id|Name|
+---+----+
|  1|   A|
|  2|   B|
+---+----+



### 28 Find missing Numbers in the DataFrame

In [31]:
data = [ (1, ), (2,), (3,), (6,), (7,), (8,)]
schema="Id int"
df = spark.createDataFrame(data,schema=schema)
df.show()

+---+
| Id|
+---+
|  1|
|  2|
|  3|
|  6|
|  7|
|  8|
+---+



In [40]:
df_list = df.select(min(col('Id')), max(col('Id')))
df_new = spark.range(df_list.first()[0], df_list.first()[1]+1)
df_new.subtract(df).show()

+---+
| id|
+---+
|  5|
|  4|
+---+



### 31

In [42]:
data=[
('Rudra','math',79),
('Rudra','eng',60),
('Shivu','math', 68),
('Shivu','eng', 59),
('Anu','math', 65),
('Anu','eng',80)
]
schema="Name string,Sub string,Marks int"
df=spark.createDataFrame(data,schema)
df.show()

+-----+----+-----+
| Name| Sub|Marks|
+-----+----+-----+
|Rudra|math|   79|
|Rudra| eng|   60|
|Shivu|math|   68|
|Shivu| eng|   59|
|  Anu|math|   65|
|  Anu| eng|   80|
+-----+----+-----+



In [50]:
df1 = df.groupBy('Name').pivot('Sub').sum('Marks')
df1.show()

+-----+---+----+
| Name|eng|math|
+-----+---+----+
|Shivu| 59|  68|
|Rudra| 60|  79|
|  Anu| 80|  65|
+-----+---+----+



In [53]:
# OR 
df2 = df.groupBy(col('Name')).agg(collect_list(col("Marks")).alias("Marks_New"))
df2.show()
df3 = df2.select(col('Name'), col('Marks_New')[0].alias('math'),col('Marks_New')[1].alias('eng'))
df3.show()

+-----+---------+
| Name|Marks_New|
+-----+---------+
|Rudra| [79, 60]|
|Shivu| [68, 59]|
|  Anu| [65, 80]|
+-----+---------+

+-----+----+---+
| Name|math|eng|
+-----+----+---+
|Rudra|  79| 60|
|Shivu|  68| 59|
|  Anu|  65| 80|
+-----+----+---+



### 43 Write a Pyspark code to get all the customer ID along with their final balance amount after calculating their transactions based on transaction type.
If any customer has not any transactions then his final balance should remain same as current amount.

In [19]:
transactions_data = [
    (1, "credit", 30.0),
    (1, "debit", 90.0),
    (2, "credit", 50.0),
    (3, "debit", 57.0),
    (2, "debit", 90.0)]
transaction_schema = StructType([
    StructField('customer_id', IntegerType(), True),
    StructField("transaction_type", StringType(), True),
    StructField("transaction_amount", FloatType(), True)
])

trans_df = spark.createDataFrame(transactions_data, schema=transaction_schema)
trans_df.show()

amounts_data = [
    (1, 1000.0),
    (2, 2000.0),
    (3, 3000.0),
    (4, 4000.0)]
amount_schema = StructType([
    StructField("customer_id", IntegerType(), True),
    StructField("current_amount", FloatType(), True)])

amt_df = spark.createDataFrame(amounts_data, schema=amount_schema)
amt_df.show()

+-----------+----------------+------------------+
|customer_id|transaction_type|transaction_amount|
+-----------+----------------+------------------+
|          1|          credit|              30.0|
|          1|           debit|              90.0|
|          2|          credit|              50.0|
|          3|           debit|              57.0|
|          2|           debit|              90.0|
+-----------+----------------+------------------+

+-----------+--------------+
|customer_id|current_amount|
+-----------+--------------+
|          1|        1000.0|
|          2|        2000.0|
|          3|        3000.0|
|          4|        4000.0|
+-----------+--------------+



In [20]:
df_1 = trans_df.groupBy('customer_id').agg(sum(when(col('transaction_type') == 'credit', col('transaction_amount'))\
                                               .otherwise(0)).alias('total_credit'),\
                                           sum(when(col('transaction_type') == 'debit', col('transaction_amount'))\
                                               .otherwise(0)).alias('total_debit'))\
                                      .withColumn('total_remaining_amnt', col('total_debit')-col('total_credit'))
df_1.show()

+-----------+------------+-----------+--------------------+
|customer_id|total_credit|total_debit|total_remaining_amnt|
+-----------+------------+-----------+--------------------+
|          1|        30.0|       90.0|                60.0|
|          2|        50.0|       90.0|                40.0|
|          3|         0.0|       57.0|                57.0|
+-----------+------------+-----------+--------------------+



In [21]:
df_2 = amt_df.join(df_1, amt_df.customer_id == df_1.customer_id, how='left')\
                          .withColumn('total_remaining_amnt', coalesce(col('total_remaining_amnt'), lit(0)))\
                          .withColumn('current_amount', col('current_amount') - col('total_remaining_amnt'))\
                          .drop(df_1.customer_id)\
                          .select('customer_id', 'current_amount')
df_2.show()

+-----------+--------------+
|customer_id|current_amount|
+-----------+--------------+
|          1|         940.0|
|          2|        1960.0|
|          3|        2943.0|
|          4|        4000.0|
+-----------+--------------+



In [54]:
spark.stop()